In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@eteproject.dfs.core.windows.net/customers")

In [0]:
df= df.drop("_rescued_data")

In [0]:
df= df.withColumn("org",split(col("email"),"@")[1])

In [0]:
display(df)

In [0]:
df.groupBy("org").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending=False).display()

In [0]:
df_gmail=df.filter(col("org")=="gmail.com")
display(df_gmail)
df_hotmail=df.filter(col("org")=="hotmail.com")
display(df_hotmail)
df_yahoo=df.filter(col("org")=="yahoo.com")
display(df_yahoo)

In [0]:
df_name=df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df= df_name.drop("first_name","last_name")
display(df)


In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@eteproject.dfs.core.windows.net/customers")

In [0]:
%sql 
create table if not exists databricks_ete.silver.customers_silver
using delta
location "abfss://silver@eteproject.dfs.core.windows.net/customers"


In [0]:
%sql
select * from databricks_ete.silver.customers_silver